# BMEB W4020: Circuits in the Brain 2020 Fall Homework \#3 Handout
*Author:* Tingkai Liu <tl2747@columbia.edu>


*Based on Previous Work by:* Tingkai Liu, Mehmet Kerem Turkcan, Chung-Heng Yeh,
                  Konstantinos Psychas <kp2547@columbia.edu>, Cong Han


Copyright 2012-2020 Tingkai Liu, Mehmet Kerem, Aurel A. Lazar and Chung-Heng Yeh, Cong Han


# PROBLEM \#1 - $\delta$-insensitive TDM

The stimulus of a single-input single-output (SISO) TEM is modeled as
a bandlimited function of the form
$$
u(t)= \sum_{k=1}^{15} u(kT) \frac{\sin \Omega (t-kT)}{\Omega (t-kT)},
$$
where $\Omega = 2 \pi \cdot 30$ Hz and $T=\frac{\pi}{\Omega}$.

Assume that the TEM describes a Leaky IAF neuron. After choosing appropriate TEM parameter values (with finite R) generate the trigger times in the time interval $[-2T, 18T]$. Suggestion: RC = 20 ms.

### Tasks
1. Implement a threshold insensitive ($\delta$-insensitive) decoding algorithm for Leaky IAF.
2. Plot the recovery error (difference between the input stimulus and the recovered waveform) and calculate the Signal-to-Noise ratio of the recovery in decibel (dB).
3. Compare the $\delta$-insensitive recovery result with the $\delta$-sensitive result (implementation of $\delta$-sensitive algorithm can be found in lecture notebook)
4.Assume that the LIF neuron generates trigger times using
  a threshold value $\delta$ whereas the recovery algorithm employs
  $\delta +\varepsilon$.  Plot the mean-square stimulus recovery error
  as a function of the threshold $\delta$ for at least thre different
  values of the parameter $\varepsilon$ (${\it e.g.,} ~\varepsilon =
  10^{-3}\delta$, $\varepsilon =
  10^{-2}\delta$ and $\varepsilon = 10^{-1} \delta$). Compare the error between $\delta$-sensitive and $\delta$-insensitive recoveries.


### Note
Signal-to-Noise ratio (SNR) of the recovery can be calculated for signal $u(t)$ and recovered signal $u_{rec}(t)$ as 
$$
SNR = 10\log_{10}\left(\frac{mean(u^2)}{mean((u-u_{rec})^2)}\right)
$$
the result will be in dB

## $\delta$-insenitive recovery (refer to Chapter \#6 for more details)
TDM algorithms are derived from t-transforms of the point neuron models. For the case of ideal IAF, this is written as 
\begin{align*}
\frac{1}{\kappa}\int_{t_k}^{t_{k+1}}u(t)+b \ dt = \delta\\
\int_{t_k}^{t_{k+1}}u(t) dt = \kappa\delta - b(t_{k+1} - t_k)\\
\end{align*}

Because the t-transform above is dependent on the threshold value $\delta$, the recovery algorithm is also depedent on the threshold $\delta$. In situations where $\delta$ is not directly observable (as is the case for most neural systems), we need to derive an alternative algorithm that does not depend on $\delta$ explicity.

To do this, we rely on the _Compensation Principle_, which is based on the observation that for a fixed value $\delta$, subtraction between 2 consecutive t-transforms remove $\delta$ completely:
$$
\int_{t_{l+1}}^{t_{l+2}} u(s) ds - \int_{t_{l}}^{t_{l+1}} u(s) ds = -b[(t_{l+2} - t_{l+1}) - (t_{l+1}-t_l)]
$$
$\forall l \in \mathbb{Z}$

### Derivation of TDM algorithm
Recall, the signals at hand are assumed to be in a functional space of bandlimited signals, which is also Reproducing Kernel Hilbert Space with sinc kernels(see Chapter 7 in book for more details). Hence, the signal to be recovered can be written as 
$$u(t) = \sum_{k \in \mathbb{Z}} c_k \cdot g(t-s_k)$$
where $g(t) = sin(\Omega t)/\pi t$ and $s_k = (t_{k} + t_{k+1})/2$.

For the $\delta$-sensitive case, substituting expression for $u(t)$ (again, under the assumption of the functional space), we have 
$$\int_{t_l}^{t_{l+1}}\sum_{k \in \mathbb{Z}} c_k \cdot g(t-s_k)dt = \kappa\delta - b (t_{l+1}-t_l)$$.
since signals are in $\mathcal{L}^{2}$, we can safely exchange the two linear operations $\int , \sum$ and obtain
$$\sum_{k \in \mathbb{Z}} c_k \underbrace{\int_{t_l}^{t_{l+1}} g(t-s_k)dt}_{[\mathbf{G}]_{lk}} = \underbrace{\kappa\delta - b (t_{l+1}-t_l)}_{q_k}$$.
The recovery of the original signal therefore amounts to _finding $c_k$_ which reduces to a matrix inversion problem:
$$\mathbf{c} = \mathbf{G}^{+}\mathbf{q}$$

For the case of $\delta$-insenstive recovery, the expression above needs to be re-written such that a difference is taken between consecutive entries of $\mathbf{q}$, and this can be done in matrix form with a toeplitz matrix of the form
$$\mathbf{B} = \begin{bmatrix} 
-1 & 1 & 0 & 0 & \cdots & 0 & 0 \\
0 & -1 & 1 & 0 & \cdots & 0 & 0 \\
\vdots & \vdots & \vdots & \vdots & \ddots & \vdots & \vdots \\
0 & 0 & 0 & 0 & \cdots & -1 & 1 \\
0 & 0 & 0 & 0 & \cdots & 0 & -1 \\
\end{bmatrix}$$

It can then be shown that the encoding
$$\int_{t_{l+1}}^{t_{l+2}} u(s) ds - \int_{t_{l}}^{t_{l+1}} u(s) ds = -b[(t_{l+2} - t_{l+1}) - (t_{l+1}-t_l)]$$
can be now re-written for the $\delta$-insensitive case as 
$$
{\bf BGc} = {\bf Bq}
$$
which gives recovery algorithm:
$$\mathbf{c} = \mathbf{B}^{-1}(\mathbf{B}\mathbf{G}\mathbf{B}^{-1})^{+}\cdot \mathbf{B}\mathbf{q}$$

# Problem 2 - Derivation of TEM/TDM algorithm for Rinzel with dendritic processing
In this problem you are asked to derive and implement a TEM/TDM algorithm for Rinzel neuron with dendritive processing modeled as a linear filter ($h(t)$ in the figure below).

Below is the encoding circuit consisting of a filter in cascade with a Hodgkin-Huxley neuron:
   <center><img src="./rinzel_dendrite.png" width=650/></center>

In the figure above, the external current injected into the neuron is given as 
$$
I_{ext}(t) = b + \underbrace{\left( u \ast h \right)(t)}_{v(t)}
$$
where $b$ is the bias current (also written as injected current $I$ in other texts).


### Tasks
1. Generate the impulse response of $h(t)$ and visualize.
2. With $b=55$, Encode a randomly generated input stimulus (from Problem 1) using a reduced PIF neuron that is equivalent to the Rinzel neuron model. 
3. Derive an algorithm to recover the signal $u(t)$ from the recieved spikes.
    1. Writing down the $t$-transform of the encoding circuit shown above in an inner product form.
    2. Find the time decoding machine (TDM) that recovers the signal $u$. Particularly, provide forms for $q_k$ and $[G]_{lk}$. Please write down the important procedures.
3. Recover the signal $u(t)$ from output spike times of the reduced PIF and show encoding error and SNR (as in Problem \#1 above)


### Stimulus and Filter
Use the stimulus $u(t)$ from Problem \#1, and the filter $h$ is 
$$
h(t)= 3 \mbox{exp}\left(-100 t\right)\left(\frac{(150 t)^3}{3!}-\frac{(150t)^5}{5!}\right) \cdot \mathbb{1}_{t\ge 0}
$$
note that $\mathbb{1}_{t\ge 0}$ is also known as the Heaviside Step function which ensures that the filter $h(t)$ is causal.

### Note
You know the filter $h$ and filtered output $v(t) = (u \ast h)(t)$, but you _do not_ know $u(t)$. You can read the spike times, and you want to recover $u(t)$ from the spikes.

In [1]:
% Start by clearing MATLAB Workspace.
close all; clearvars; clc;

In [2]:
% Continue with signal generator definitions.


In [3]:
%plot -b inline -f svg -r 96 -s 1080,720

In [4]:
% Let us start by defining globals and initializing the input stimulus.
rng(20204020);                               % Fix the RNG
% TODO: generate input
signal_generator = nan;
t = nan;   % Time
U = nan;   % Signal

# Problem 1

In [6]:
% TODO: Encod input U with LIF 
delta = 0.008;                            % set delta [mV]
kappa = 1;                                % set kappa [mF]
bias  = kappa*delta*Omega/pi+1;           % set bias  [mu A]

In [11]:
% TODO: Recovery 

B = nan;
q = nan;
G = nan;

c_insens = nan; % delta-insensitive recovery coefficients
c_sens = nan; % delta-sensitive recovery coefficients

In [4]:
% TOOD: PLOT Encoding
% Suggestion: 3 rows, top) input, middle) output voltage and spike time, bottom) nyquist rate vs. inter-sample interval

In [15]:
% Compute SNR
SNR_f = @(u, u_rec) 10*log10(mean(u.^2)/mean((u-u_rec).^2));

In [5]:
% TOOD: PLOT Recovery

In [17]:
% TODO recovery with noisy threshold
noise_ratio = nan;
u_rec_sens_noisy = zeros(numel(t), length(noise_ratio));
u_rec_insens_noisy = zeros(numel(t), length(noise_ratio));
for n = 1:length(noise_ratio)
    noise = noise_ratio(n)*delta;
    B_noisy = nan;
    q_noisy = nan;
    G_noisy = nan;
    c_sens_noisy = nan;
    c_insens_noisy = nan;

    % Recover the stimulus
    u_rec_insens_noisy(:, n) = nan;
    u_rec_sens_noisy(:, n) = nan;
 end

In [6]:
% TODO: plot

# Problem 2

In [7]:
% TODO: choose bias, use any value that makes rinzel model spike
bias=nan;

In [8]:
% Generate a filter h, with a
% temporal support on the interval [T_1, T_2]. The filter generated here
% will be repeatedly used through this demo.

T_1 = 0; T_2 = 0.1;                           % specify T_1 and T_2
t_filt = T_1:dt:T_2;                          % set the length of the impulse response, [s]
a = 150;                                      % set the filter parameter
h = 3*a*exp(-a*t_filt).*((a*t_filt).^3/...    % set the filter impulse response
    factorial(3)-(a*t_filt).^5/factorial(5));

Error using eval
Unrecognized function or variable 'dt'.



In [21]:
% TOOD: filter signal
v = nan;

In [23]:
% TODO: compute PRC at given biase
prc = nan;

In [24]:
% TODO: Compute PIF

In [9]:
% TODO: plot Rinzel and PIF encoding result

In [ ]:
% TODO: describe the algorithm for recovering the signal u(t) given h(t)

In [30]:
% TODO: recover u(t)

In [10]:
% TODO: plot recovery result. The recovery may be bad at the boundary because all signals are finite, 
%  you can focus on the middle part of the signal for comparison.